In [ ]:
!apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
!pip install gradio diffusers moviepy pytesseract


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 121.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.4 MB/s eta 0:00:00


In [ ]:
!pip install moviepy


In [ ]:
# Reset permissions in Colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `personal` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `personal`

In [ ]:
torch.cuda.empty_cache()

NameError: name 'torch' is not defined

In [ ]:
import gradio as gr
import torch
import tempfile
import moviepy.editor as mpy
import numpy as np
from diffusers import StableDiffusionPipeline
from transformers import PaliGemmaForConditionalGeneration, AutoProcessor
from PIL import Image

# Load models
MODEL_PATH = "/content/drive/MyDrive/paligemma_finetuned"
model = PaliGemmaForConditionalGeneration.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16,
    device_map="auto",
    local_files_only=True
)
processor = AutoProcessor.from_pretrained(MODEL_PATH, local_files_only=True)

sd_pipeline = StableDiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1",
    torch_dtype=torch.float16
).to("cuda")

def answer_question(image, question):
    try:
        # Handle missing image
        if image is None:
            image = Image.new("RGB", (224, 224), (255, 255, 255))
        else:
            image = image.convert("RGB")

        # Process inputs
        inputs = processor(
            text=f"<image> {question}",
            images=image,
            return_tensors="pt",
            truncation=True,
            max_length=768,
            padding="max_length"
        ).to("cuda")

        # Generate answer
        outputs = model.generate(**inputs, max_new_tokens=100)
        answer_text = processor.decode(outputs[0], skip_special_tokens=True)

        # Generate image
        gen_image = sd_pipeline(
            prompt=f"Illustration: {answer_text[:50]}",
            height=512,
            width=512,
            num_inference_steps=25
        ).images[0]

        # Create video
        video_path = tempfile.mktemp(suffix=".mp4")
        frames = [np.array(gen_image)] * 3  # Convert to numpy arrays
        clip = mpy.ImageSequenceClip(frames, fps=1)
        clip.write_videofile(video_path, verbose=False, logger=None)

        return answer_text, gen_image, video_path

    except Exception as e:
        return f"Error: {str(e)}", None, None

interface = gr.Interface(
    fn=answer_question,
    inputs=[
        gr.Image(label="Upload Image (optional)", type="pil"),
        gr.Textbox(label="Question")
    ],
    outputs=[gr.Textbox(), gr.Image(), gr.Video()],
    title="NCERT Tutor"
)

interface.launch(share=True)

  if event.key is 'enter':



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



model_index.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

Fetching 28 files:   0%|          | 0/28 [00:00<?, ?it/s]

...

(…)ion_pytorch_model.safetensors.index.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/809 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/4 [00:00<?, ?it/s]

ValueError: Pipeline <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> expected {'unet', 'tokenizer', 'image_encoder', 'text_encoder', 'feature_extractor', 'safety_checker', 'vae', 'scheduler'}, but only {'vae', 'tokenizer', 'scheduler', 'text_encoder'} were passed.

In [ ]:
#llamakesath
!pip install diffusers transformers gradio moviepy torch accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import gradio as gr
import torch
import tempfile
import moviepy.editor as mpy
import numpy as np
from diffusers import StableDiffusion3Pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from PIL import Image
import pytesseract


instruct_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct", trust_remote_code=True)
instruct_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
).to("cuda")


sd_pipeline = StableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3.5-medium",
    torch_dtype=torch.bfloat16
)
sd_pipeline = sd_pipeline.to("cuda")

def answer_question(image, question):
    try:

        if image is None:
            image = Image.new("RGB", (224, 224), (255, 255, 255))
            extracted_text = ""
            combined_prompt = f"Question: {question}\nAnswer:"
        else:
            image = image.convert("RGB")

            extracted_text = pytesseract.image_to_string(image)
            if not extracted_text.strip():
                extracted_text = "No text detected in the image."

            combined_prompt = (
                f"Image context: {extracted_text}\n"
                f"Question: {question}\n"
                f"Answer concisely in context of the image:"
            )


        inputs = instruct_tokenizer(combined_prompt, return_tensors="pt").to("cuda")
        prompt_length = inputs["input_ids"].shape[1]

        outputs = instruct_model.generate(**inputs, max_new_tokens=2048)
        generated_tokens = outputs[0][prompt_length:]
        answer_text = instruct_tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()


        refined_prompt = (
            f"Create a highly detailed, educational illustration that visually explains the following answer: '{answer_text}'. "
            f"Focus on the key elements and context of the answer and include clear, legible text annotations with high contrast. "
            f"Ensure all textual elements are crisp and easily readable."
        )
        gen_image = sd_pipeline(
            prompt=refined_prompt,
            height=768,
            width=768,
            num_inference_steps=70,
            guidance_scale=7.5
        ).images[0]


        frames = []
        seeds = [42, 123, 987]
        for seed in seeds:
            generator = torch.Generator("cuda").manual_seed(seed)
            result = sd_pipeline(
                prompt=refined_prompt,
                height=768,
                width=768,
                num_inference_steps=70,
                guidance_scale=7.5,
                generator=generator
            )
            frames.append(np.array(result.images[0]))
        video_path = tempfile.mktemp(suffix=".mp4")
        clip = mpy.ImageSequenceClip(frames, fps=1)
        clip.write_videofile(video_path, codec="libx264", audio=False)

        return answer_text, gen_image, video_path

    except Exception as e:
        return f"Error: {str(e)}", None, None


interface = gr.Interface(
    fn=answer_question,
    inputs=[
        gr.Image(label="Upload Image (optional)", type="pil"),
        gr.Textbox(label="Question")
    ],
    outputs=[
        gr.Textbox(label="Answer"),
        gr.Image(label="Generated Illustration"),
        gr.Video(label="Generated Video")
    ],
    title="Virtual Tutor"
)

interface.launch(share=True)


  if event.key is 'enter':

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://480e79d8618f3814ad.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#mistralwala
torch.cuda.empty_cache()

In [ ]:
import gradio as gr
import torch
import tempfile
import moviepy.editor as mpy
import numpy as np
from diffusers import StableDiffusionXLPipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from PIL import Image

# -------------------------------
# Load the Best Text Generation Model
# -------------------------------
# Using Mistral-7B-Instruct for detailed and accurate text generation.
mistral_tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    trust_remote_code=True
)
mistral_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# -------------------------------
# Load the Best Image Generation Model
# -------------------------------
# Using Stable Diffusion XL Turbo for state-of-the-art text-to-image synthesis.
sd_pipeline = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/sdxl-turbo",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

def answer_question(image, question):
    try:
        # If no image is provided, use a default white image and caption.
        if image is None:
            image_caption = "A plain white image."
            image = Image.new("RGB", (224, 224), (255, 255, 255))
        else:
            image = image.convert("RGB")
            image_caption = "An uploaded image."  # Optionally, you could use an image captioning model here.

        # Build a prompt for the text generation model that includes the image caption.
        prompt = f"Based on the following image description: {image_caption}\nQuestion: {question}\nAnswer:"

        # Tokenize the prompt and move tensors to GPU.
        inputs = mistral_tokenizer(prompt, return_tensors="pt").to("cuda")

        # Generate the answer using Mistral.
        outputs = mistral_model.generate(**inputs, max_new_tokens=100)
        answer_text = mistral_tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Use the generated answer (first 50 characters) to form an image-generation prompt.
        gen_image = sd_pipeline(
            prompt=f"Illustration: {answer_text[:50]}",
            height=512,
            width=512,
            num_inference_steps=25
        ).images[0]

        # Create a simple video by repeating the generated image for 3 frames.
        video_path = tempfile.mktemp(suffix=".mp4")
        frames = [np.array(gen_image)] * 3
        clip = mpy.ImageSequenceClip(frames, fps=1)
        clip.write_videofile(video_path, verbose=False, logger=None)

        return answer_text, gen_image, video_path

    except Exception as e:
        return f"Error: {str(e)}", None, None

# -------------------------------
# Build the Gradio Interface
# -------------------------------
interface = gr.Interface(
    fn=answer_question,
    inputs=[
        gr.Image(label="Upload Image (optional)", type="pil"),
        gr.Textbox(label="Question")
    ],
    outputs=[
        gr.Textbox(label="Answer"),
        gr.Image(label="Generated Illustration"),
        gr.Video(label="Generated Video")
    ],
    title="NCERT Tutor"
)

interface.launch(share=True)


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f3d6b20e3cec78b8ad.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#debugggggggggggggggggzyadaaccurate

In [ ]:
import gradio as gr
import torch
import tempfile
import moviepy.editor as mpy
import numpy as np
from diffusers import StableDiffusionXLPipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from PIL import Image

# -------------------------------
# Load the Best Text Generation Model
# -------------------------------
# Using Mistral-7B-Instruct-v0.3 for detailed and accurate text generation.
mistral_tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    trust_remote_code=True
)
mistral_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# -------------------------------
# Load the Best Image Generation Model
# -------------------------------
# Using Stable Diffusion XL Turbo for state-of-the-art text-to-image synthesis.
sd_pipeline = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/sdxl-turbo",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

def answer_question(image, question):
    try:
        # If no image is provided, use a default white image and caption.
        if image is None:
            image_caption = "A plain white image."
            image = Image.new("RGB", (224, 224), (255, 255, 255))
        else:
            image = image.convert("RGB")
            image_caption = "An uploaded image."  # Optionally, you could use an image captioning model here.

        # Build a prompt for the text generation model that includes the image caption.
        prompt = f"Based on the following image description: {image_caption}\nQuestion: {question}\nAnswer:"

        # Tokenize the prompt and move tensors to GPU.
        inputs = mistral_tokenizer(prompt, return_tensors="pt").to("cuda")

        # Generate the answer using Mistral with more tokens for a complete answer.
        outputs = mistral_model.generate(**inputs, max_new_tokens=256)
        answer_text = mistral_tokenizer.decode(outputs[0], skip_special_tokens=True)

        # For image generation, create a refined prompt.
        # Here, we use the first 200 characters of the answer (if available) to craft an "accurate" illustration prompt.
        refined_prompt = f"Accurate educational illustration: {answer_text[:200]}"

        # Generate an illustration image with increased inference steps and guidance scale.
        gen_image = sd_pipeline(
            prompt=refined_prompt,
            height=512,
            width=512,
            num_inference_steps=50,
            guidance_scale=7.0
        ).images[0]

        # Generate a video by creating 3 unique frames using different random seeds.
        frames = []
        seeds = [42, 123, 987]
        for seed in seeds:
            generator = torch.Generator("cuda").manual_seed(seed)
            result = sd_pipeline(
                prompt=refined_prompt,
                height=512,
                width=512,
                num_inference_steps=50,
                guidance_scale=7.0,
                generator=generator
            )
            frames.append(np.array(result.images[0]))

        # Assemble the frames into a video (each frame displayed for 1 second).
        video_path = tempfile.mktemp(suffix=".mp4")
        clip = mpy.ImageSequenceClip(frames, fps=1)
        clip.write_videofile(video_path, verbose=False, logger=None)

        return answer_text, gen_image, video_path

    except Exception as e:
        return f"Error: {str(e)}", None, None

# -------------------------------
# Build the Gradio Interface
# -------------------------------
interface = gr.Interface(
    fn=answer_question,
    inputs=[
        gr.Image(label="Upload Image (optional)", type="pil"),
        gr.Textbox(label="Question")
    ],
    outputs=[
        gr.Textbox(label="Answer"),
        gr.Image(label="Generated Illustration"),
        gr.Video(label="Generated Video")
    ],
    title="Virtual Tutor"
)

interface.launch(share=True)


  if event.key is 'enter':

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e5bcf37406ffc2e2d8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#mistraltextextractionwala

In [ ]:
import gradio as gr
import torch
import tempfile
import moviepy.editor as mpy
import numpy as np
from diffusers import StableDiffusionXLPipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from PIL import Image
import pytesseract  # For OCR text extraction

# -------------------------------
# Load the Instruction-Tuned Language Model
# -------------------------------
# Mistral-7B-Instruct-v0.3 is used for more accurate and refined answers.
instruct_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")
instruct_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    torch_dtype=torch.float16
).to("cuda")

# -------------------------------
# Load the Image Generation Model
# -------------------------------
# Using Stable Diffusion XL Turbo for high-quality text-to-image synthesis.
sd_pipeline = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/sdxl-turbo",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

def answer_question(image, question):
    try:
        # Ensure the image is in RGB format; create a default if none is provided.
        if image is None:
            image = Image.new("RGB", (224, 224), (255, 255, 255))
        else:
            image = image.convert("RGB")

        # -------------------------------
        # Stage 1: OCR Text Extraction
        # -------------------------------
        # Extract text from the image using pytesseract.
        extracted_text = pytesseract.image_to_string(image)
        if not extracted_text.strip():
            extracted_text = "No text detected in the image."

        # -------------------------------
        # Stage 2: Answer Generation with Mistral-7B-Instruct
        # -------------------------------
        # Combine the extracted text and the user's question to form a detailed prompt.
        combined_prompt = f"Extracted text from image: {extracted_text}\nQuestion: {question}\nAnswer:"
        inputs = instruct_tokenizer(combined_prompt, return_tensors="pt").to("cuda")

        # Record the prompt token length.
        prompt_length = inputs["input_ids"].shape[1]
        # Increase max_new_tokens to get a full-length answer.
        outputs = instruct_model.generate(**inputs, max_new_tokens=300)
        # Decode only the tokens that were generated beyond the prompt.
        generated_tokens = outputs[0][prompt_length:]
        answer_text = instruct_tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

        # -------------------------------
        # Stage 3: Image Generation (Illustration)
        # -------------------------------
        # Create a refined prompt that ties the question and answer together.
        # This prompt is designed to generate an illustration that visually explains the answer in context.
        refined_prompt = (
            f"Create a detailed educational illustration that explains the following question: '{question}'. "
            f"Include key elements from the answer: '{answer_text[:200]}'. "
            f"Make the image clear, informative, and visually engaging."
        )
        gen_image = sd_pipeline(
            prompt=refined_prompt,
            height=512,
            width=512,
            num_inference_steps=50,
            guidance_scale=7.0
        ).images[0]

        # -------------------------------
        # Stage 4: Video Generation
        # -------------------------------
        # Create a short video by generating multiple frames with different random seeds.
        frames = []
        seeds = [42, 123, 987]
        for seed in seeds:
            generator = torch.Generator("cuda").manual_seed(seed)
            result = sd_pipeline(
                prompt=refined_prompt,
                height=512,
                width=512,
                num_inference_steps=50,
                guidance_scale=7.0,
                generator=generator
            )
            frames.append(np.array(result.images[0]))

        # Assemble the frames into a video (each frame displayed for 1 second).
        video_path = tempfile.mktemp(suffix=".mp4")
        clip = mpy.ImageSequenceClip(frames, fps=1)
        clip.write_videofile(video_path, codec="libx264", audio=False)

        return answer_text, gen_image, video_path

    except Exception as e:
        return f"Error: {str(e)}", None, None

# -------------------------------
# Build the Gradio Interface
# -------------------------------
interface = gr.Interface(
    fn=answer_question,
    inputs=[
        gr.Image(label="Upload Image (optional)", type="pil"),
        gr.Textbox(label="Question")
    ],
    outputs=[
        gr.Textbox(label="Answer"),
        gr.Image(label="Generated Illustration"),
        gr.Video(label="Generated Video")
    ],
    title="Virtual Tutor"
)

interface.launch(share=True)


  if event.key is 'enter':

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a8d19b271b4cc5e7fc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#stablediffusion3.5accuracymore

In [ ]:
import gradio as gr
import torch
import tempfile
import moviepy.editor as mpy
import numpy as np
from diffusers import StableDiffusion3Pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from PIL import Image
import pytesseract  # For OCR text extraction

# -------------------------------
# Load the Instruction-Tuned Language Model
# -------------------------------
# Using Mistral-7B-Instruct-v0.3 for refined answer generation.
instruct_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")
instruct_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    torch_dtype=torch.float16
).to("cuda")

# -------------------------------
# Load the Image Generation Model using StableDiffusion3Pipeline
# -------------------------------
# Using "stabilityai/stable-diffusion-3.5-medium" for image generation.
sd_pipeline = StableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3.5-medium",
    torch_dtype=torch.bfloat16
)
sd_pipeline = sd_pipeline.to("cuda")

def answer_question(image, question):
    try:
        # Ensure the image is in RGB format; if none is provided, create a default white image.
        if image is None:
            image = Image.new("RGB", (224, 224), (255, 255, 255))
            extracted_text = ""
            combined_prompt = f"Question: {question}\nAnswer:"  # No image context
        else:
            image = image.convert("RGB")
            # -------------------------------
            # Stage 1: OCR Text Extraction
            # -------------------------------
            extracted_text = pytesseract.image_to_string(image)
            if not extracted_text.strip():
                extracted_text = "No text detected in the image."
            # -------------------------------
            # Stage 2: Answer Generation with Image Context
            # -------------------------------
            combined_prompt = (
                f"Image context: {extracted_text}\n"
                f"Question: {question}\n"
                f"Answer concisely in context of the image:"
            )

        # Tokenize the prompt and move tensors to GPU.
        inputs = instruct_tokenizer(combined_prompt, return_tensors="pt").to("cuda")
        prompt_length = inputs["input_ids"].shape[1]
        # Increase max_new_tokens to 1000 to generate longer answers.
        outputs = instruct_model.generate(**inputs, max_new_tokens=1000)
        generated_tokens = outputs[0][prompt_length:]
        answer_text = instruct_tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

        # -------------------------------
        # Stage 3: Refined Image Generation (Illustration)
        # -------------------------------
        # Build a refined prompt that instructs the model to create an illustration that accurately explains the answer.
        refined_prompt = (
            f"Create a highly detailed, educational illustration that explains the following answer: '{answer_text}'. "
            f"Focus on key elements and context from the answer to make the image informative and visually engaging."
        )
        gen_image = sd_pipeline(
            prompt=refined_prompt,
            height=768,
            width=768,
            num_inference_steps=70,
            guidance_scale=7.5
        ).images[0]

        # -------------------------------
        # Stage 4: Video Generation
        # -------------------------------
        # Generate three unique frames using different random seeds.
        frames = []
        seeds = [42, 123, 987]
        for seed in seeds:
            generator = torch.Generator("cuda").manual_seed(seed)
            result = sd_pipeline(
                prompt=refined_prompt,
                height=768,
                width=768,
                num_inference_steps=70,
                guidance_scale=7.5,
                generator=generator
            )
            frames.append(np.array(result.images[0]))
        video_path = tempfile.mktemp(suffix=".mp4")
        clip = mpy.ImageSequenceClip(frames, fps=1)
        clip.write_videofile(video_path, codec="libx264", audio=False)

        return answer_text, gen_image, video_path

    except Exception as e:
        return f"Error: {str(e)}", None, None

# -------------------------------
# Build the Gradio Interface
# -------------------------------
interface = gr.Interface(
    fn=answer_question,
    inputs=[
        gr.Image(label="Upload Image (optional)", type="pil"),
        gr.Textbox(label="Question")
    ],
    outputs=[
        gr.Textbox(label="Answer"),
        gr.Image(label="Generated Illustration"),
        gr.Video(label="Generated Video")
    ],
    title="Virtual Tutor: Detailed Answer & Illustration"
)

interface.launch(share=True)


  if event.key is 'enter':

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/809 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://68d7f989301c9a8efb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
